This notebook can be used to load raw ultraleap data, 
save cleaned dataframes for each block, 
and generate dataframes of distances for further feature extraction

Import public packages and functions

In [511]:
import os
import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from itertools import compress
import seaborn as sn

import openpyxl
from datetime import datetime
import math
import statistics as stat
import json

In [512]:
def get_repo_path_in_notebook():
    """
    Finds path of repo from Notebook.
    Start running this once to correctly find
    other modules/functions
    """
    path = os.getcwd()
    repo_name = 'ultraleap_analysis'

    while path[-len(repo_name):] != 'ultraleap_analysis':

        path = os.path.dirname(path)

    return path


In [513]:
repo_path = get_repo_path_in_notebook()
code_path = os.path.join(repo_path, 'code')
os.chdir(code_path)

Import own functions

In [514]:
import import_data.import_and_convert_data as import_dat
import import_data.find_paths as find_paths
import import_data.preprocessing_meta_info as meta_info
import sig_processing.segment_tasks as seg_tasks
import movement_calc.helpfunctions as hp
import feature_extraction.get_features as get_feat
import feature_extraction.get_features as get_files
import feature_extraction.get_files as get_files
import model_prediction.func_factory as ff

In [515]:
importlib.reload(get_files)

<module 'feature_extraction.get_files' from '/Users/arianm/Documents/GitHub/ultraleap_analysis/code/feature_extraction/get_files.py'>

### 1. Loading Features (X) and scores (y)

Load features from csv and scores/ lables from table

In [516]:
def loading_features_and_scores(task= str):

    task = task.lower()

    feat_df_path = os.path.join(repo_path, 'data', 'features', 'dataframes', 'patientdata')

    if task == 'ft':
        df = pd.read_csv(os.path.join(feat_df_path, 'ft_block_features.csv'), index_col=0)
    elif task == 'oc':
        df = pd.read_csv(os.path.join(feat_df_path, 'oc_block_features.csv'), index_col=0)

    X_df, y = X_df, y = ff.load_labels_and_adjust_feature_df(df)

    y = np.array(y)

    return X_df, y


In [517]:
X_df, y = loading_features_and_scores('ft')

In [518]:
## plotting feature boxplots based on score
# X_df.insert(1, 'scores', y)
# X = X_df[:, 1:]
# df0, df1, df2, df3, df4 =split_dataframe_by_scores(X, 'scores')
# dfs = [df0, df1, df2, df3, df4]
# boxplot_by_dataframe(dfs, 'num_events')

### 2. Select which features you want to use 

In [519]:
X = X_df.iloc[:, 1:]

## Make binary classification problem

In [520]:
y_binary = y > 1

y = y_binary
y = y.astype(int)

In [521]:
classifiers = ['linear', 'nonlinear', 'logregression', 'random forest', 'knearest', 'gaussian naive bayes']